# 1 - Redes recurrentes orientadas al procesamiento de lenguaje natural

**Sumario**

1. Introducción
2. Procesamiento de lenguaje natural
3. Redes neuronales recurrentes (RNN)
4. Encoders - Decoders
5. Modelos de atención
6. Transformers
7. Construcción de una red recurrente

## 1.1 - Introducción

En los capítulos anteriores hemos estudiado modelos de razonamiento basados en redes neuronales capaces de predecir, por ejemplo,  si un cierto animal se encuentra en una fotografía o si una casa de segunda mano se puede vender por un determinado precio.

En ambos casos, nuestros modelos trataban de identificar patrones durante la fase de entrenamiento mediante la asuncion de que las instancias  de entrenamiento eran individuales, independientes e identicamente distribuidas (**i.i.d**). Es decir, que **no había concepto de tiempo**.

No obstante, hay determinados procesos donde esta asunción no se cumple, ya que la información de que cada una de ellas representa depende considerablemente de las instancias anteriores y posteriores desde una perspectiva temporal. Por ejemplo, cuando estamos utilizando el lenguaje natural, el significado de cada una de las palabras que usamos depende en gran medida tant de las palabras anteriores, como de las posteriores e, incluso, del contexto o de la entonación de la persona que está hablando.

El lenguaje natural no es la única área donde tiene importancia el tiempo, otras tareas de ML incluyen:
* Composición musical
* Predicción de precio en el stock Market
* Conducción automática de vehiculos 
* etc.

Con todo, entre todas las áreas donde las redes neuronales recurrentes han destacado significativamente, sobresale el área del **procesamiento de lenguaje natural** debido a su relación directa el ser humano y el conocimiento. 

El procesamiento de lenguaje natural es un área relativamente moderna dada la dificulta que supone interpretar un texto complejo o una conversación. A pesar de esto, ha evolucionado a una velocidad pasmosa, sobretodo gracias a las redes neuronales.

Durante los primeros años, los diferentes métodos que se utilizaban estaban totalmente orientados al análisis de textos mediante **"bolsas de palabras"** (i.e., *Bag of words*, BOW por sus singlas en inglés). Así, cada documento equivalía a una bolsa y se utilizaba como entrada del algoritmo de aprendizaje con el objetivo de construir modelos de identificación. En aquel momento, las tareas principales de investigación eran:
* Búsqueda de textos en documentos.
* Agrupación de documentos similares.
* Clasificación de textos (e.g., noticias, historia, positivo-negativo, etc.).

Con la aparición de las redes neuronales recurrentes y de las capas tipo *embedding* (representaciones númericas del texto en forma matricial), se expandió enormemente el campo, permitiendo no solo mejores resultados en texto escrito, sino también el procesamiento de lenguaje natural hablado.

Ya en 2017, surge una nueva arquitectura de red neuronal que pone el campo patas arriba denominada ***transformer***. Donde se sustituyen las capas de tipo recurrente por un nuevo tipo denominado **capa de atención**. Esta capa permite codificar cada palabra en función del resto de la frase o secuencia **sin incurrir en los problemas de "olvido"** que muestran las redes recurrentes. Para ello, uso embeddings contextuales, que introducen el contexto en la representación matemática del texto dentro de la red.

A lo largo de este capítulo, describiremos aquellos conceptos básicos asociados al procesamiento del lenguaje natural que son necesarios para la construcción de redes de neuronas profundas de tipo recurrente. A continuación, nos ocuparemos del concepto de redes recurrentes e identificaremos los diferentes tipos de capas (recurrentes, LTSM, GRU, atención), así como las distintas arquitecturas destinadas a la construcción de modelos que permitan procesar series temporales y texto tanto hablado como escrito.

## 1.2 - Procesamiento de lenguaje natural

El procesamiento del lenguaje natural (*Natural Language Processing*, NLP por sus siglas en inglés) es un área de la inteligencia artificial centrada en crear sistemas capaces de interacturar con seres humanos en su mismo lenguaje, tanto de manera escrita como hablada (en este sentido, el lenguaje hablado debe ser primero transformado a lenguaje escrito mediante técnicas de reconocimiento de sonido).

Por tanto el área de NLP surge como la combinación de la lingüística y la inteligencia artificial, dando lugar a dos linear de trabajo:

* **Generación de lenguaje natural** (*Natural Language Generation*, NLG por sus siglas en inglés).
* **Entendimiento de lenguaje natural** (*Natural Language Understanding*, NLU por sus siglas en inglés). 

El procesamiento de lenguaje natural se aplica en multitud de aplicaciones:
* Elaboración de resúmenes de texto.
* Sugerencias de palabras en los procesos de escritura para mensajes o emails.
* Traducción de textos.
* Reconocimiento de entidades como marcas, lugares, empresas, celebridades, etc.
* Análisis de sentimientos con el objetivo de identificar el "tono" de un texto (e.g., positivo o negativo)
* etc.

El procesamiento del lenguaje nautural comenzó a aplicarse mediante enfoques basados en reglas definidas por expertos en lingüística. Pero como hemos visto en capítulos anteriores, las redes neuronales necesitan representaciones numéricas. De manera que se desarollaron dos enfoques capaces de representar la información textual de manera numérica.

**Enfoque léxico**, donde la información era modelada según su frecuencia de aparición en el texto. Esto puede ser de manera individual o agrupando grupos de palabras (i.e., *bag of words*).  En tal caso, se hace algo similar a una **codificación one-hot que asocia el grupo con frecuencia**.

**Enfoque semántico**, donde la información se modela en base a un **vector numérico multidimensional** que describe cada palabra a partir de unas "características" identificadas por una red neuronal.

### 1.2.1 - Bags of words

El proceso de generación de una bolsa de palabras, se fundamente en el siguiente proceso:

1. Se realiza una **"tokenización" del documento**, es decir, se extraen todas las palabras del documento (o subpalabras).

2. Se aplica un método de **conteo de palabras**. Dicho resultado puede ser normalizado. Por ejemplo mediante el uso de la técnica TF-IDF (Term Frequency - Inverse Document Frequency), que ajusta la frecuencia de un término (TF) mediante su frecuencia en **todos** los documentos. De tal forma que aquellas palabras que aparecen mucho en un documento pero también aparecen en otros documentos no tienen tanto importancia (e.g., "el", "la", "yo", etc.)

Para explicar brevemente esta técnica, consideremos que tenemos 3 "documentos" que representan reviews sobre películas:

* **Review 1:** This movie is very scary and long
* **Review 2:** This movie is not scary and is slow
* **Review 3:** This movie is spooky and good

<img src="images_1/tf_idf_example.png" width="500" data-align="center">

$$
\begin{align*}
\text{TF}_{d}(t) &= \frac{N_{d}(t)}{N_{d}} \\
\text{IDF}(t) &= \log \frac{D}{D(t)} \\
\text{TF-IDF}(t,d) &= \text{TF}_{d}(t) * \text{IDF}(t)
\end{align*}
$$

donde:
* $N_{d}(t)$ representa el número de apariciones del término $t$ en el documento $d$.
* $N_{d}$ representa el número total de términos en el documento $d$.
* $D$ representa el número total de documentos.
* $D(t)$ representa el número de documentos con el término $t$.

Inicialmente, este tipo de técnica se convirtió en una buena solución, pero tiene dos principales limitaciones:
* No se recoge ningún tipo de información relativa al orden o al **contexto de las palabras** dentro del documento.
* La **representación *sparse*** resultante aumenta cuadráticamente con respecto al tamaño del vocabulario y el número de documentos, lo que lo hace **computacionalmente problemático**. 

### 1.2.2 - Embeddings

Con el objetivo de eliminar las limitaciones que presentaba el sistema basado en bolsas de palabras, tanto en lo que se refiere a la calidad de la información como a su tamaño de representación se desarrolló el sistema de *embeddings*.

Esta técnica busca construir una representación más compacta (i.e., densa) que nos permita reflejar mejor la información, así como las relaciones entre las diferentes palabras.

De esta forma, se asigna a cada palabra (independientemente a que documento pertenezca) un vector continuo de $n$ dimensiones. La distancia existente en este espacio vectorial entre palabras denota similitud semántica.

Por ejemplo, imaginemos que tenemos un embedding de 50 dimensiones con un vocabulario de 20.000 palabras en inglés. Entre ellas podria estar la palabra *king* con el siguiente vector de valores que oscilan entre $1.6$ y $-1.6$:

<img src="images_1/embedding_king.jpg" width="700" data-align="center">

Existen diversas técnicas para la construcción de embeddings. La mayor parte de ellas utilizan aprendizaje automático con redes neuronales. En este sentido, **el objetivo es "rellenar" la matriz de valores de tamaño $V * n$**, donde $V$ es el tamaño del vocabulario y $n$ el número de dimensiones a considerar.

Una técnica muy conocida es ***Word2Vec***, la cual consiste en una red neuronal con dos capas:
* Una capa de proyección (i.e., el *embedding*) que representa la matriz anterior. Inicialmente con valores aleatorios.
* Una capa densa con una función de activación *softmax*. El número de conexiones es igual al número de dimensiones en nuestra capa de embedding, es decir, $n$.

La manera en la cual se aprende esta red neuronal es mediante una tecnica de predicción de palabras en el corpus. Es decir, partimos de una o varias palabras del texto y queremos que el modelo prediga una o varias palabras cercanas.

Existen dos arquitecturas específicas del tipo ***Word2Vec***, dependiendo de la entrada y del tipo específico de salida que queramos generar:

* [**Bolsa de palabras continua (CBOW)**](https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-cbow.html). Se predice una palabra a partir del contexto. Queremos que el modelo modifique los valores del embedding de tal manera que pueda predecir correctamente cual es la palabra del vocabulario a partir de sus palabras anteriores y posteriores.

* [***Skip-gram***](https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-skip-gram.html). Funciona de manera opuesta a CBOW, ya que nos permite predecir el contexto a partir de un determinado término.

<img src="images_1/word2vec_architectures.png" width="500" data-align="center">


<table>
    <tr>
        <th>Ejemplo CBOW</th>
    </tr>
    <tr>
        <td><img src="images_1/cbow.jpg" width="700" data-align="center">></td>
    </tr>
</table>

<table>
    <tr>
        <th>Ejemplo Skip-gram</th>
    </tr>
    <tr>
        <td><img src="images_1/skipgram_example.png" width="300" data-align="center"></td>
    </tr>
</table>

Presentar un ejemplo de skip-gram es más complicado ya que es menos visual. En este caso, hemos definido un tamaño de contexto de 1, de manera que dada una palabra, predecimos si otra tiene sentido como "contexto" de la primera.

----

**Nota:** [Google ofrece modelos ya entrenados de Word2Vec que contienen embeddings de 300 dimensiones para tres millones palabras y frases](https://code.google.com/archive/p/word2vec/).

----

Además de Word2Vec existen otros modelos similares:

* **GloVe**. Se basa en el conteo. De tal manera que se determina cuántas veces aparece una palabra en un contexto específico.
* **LexVec**. Combina GloVe y Word2Vec. Existen diferentes versiones ya que ha ido evolucionando con el tiempo (a diferencia de los anteriores).

Poco a poco, los sistemas de embedding comenzaron a utilizarse no solo por su cuenta (para medir la distancia entre palabras) sino también como capas de redes neuronales más complejas, lo que supuso una mejora en las tareas de NLP, especialmente cuando dichos embeddings ya estaban pre-entrenados (como el caso propuesto por Google).

## 1.3 - Redes neuronales recurrentes

Las redes neuronales recurrentes (*Recurrent Neural Networks*, RNNs por sus siglas en inglés) son un tipo de redes de neuronas que contienen, al menos, un ciclo dentro de sus conexiones de red. Es decir, **algunas de las neuronas de la red utilizan su propia salida o la salida de neuronas anteriores a ellas en la estructura de la red**.

De esta forma, **la información ya no fluye en una única dirección**, como observábamos en las redes de tipo *feed-forward*, sino en ambos sentidos, hacia delante y parcialmente hacia atrás.

La utilización de los procesos de retroalimentación de la salida permite a este tipo de redes contar con **ciertas características de "memoria"** en tareas de NLP.

En base a su complejidad, podemos dividir las RNNs en dos grandes categorías:

* RNNs simples.
* RNNs complejas.

### 1.3.1 - Redes recurrentes simples

Las redes recurrentes simples poseen una retroalimenación sencilla ya que solo incluyen el resultado de la neurona inmediatamente anterior.

En la siguiente figura se muestra el funcionamiento de una red recurrente representada de manera compacta y desenrollada para cada instante de tiempo $t$:

<img src="images_1/rnn_simple.png" width="500" data-align="center">

Por tanto, para el instante de tiempo $t=0$, la neurona solo utiliza como entrada la información externa, pero a partir de instante de tiempo $t=1$, la neurona utiliza como entrada la información externa como la salida para el instante de tiempo $t-1$.

A continuación se presenta una versión esquematizada de una neurona recurrente simple:

<img src="images_1/neurona_simple.png" width="300" data-align="center">

Como se puede observar en la siguiente figura, la neurona ya no solo usa la información de entrada, sino tambien el resultado obtenido para la entrada anterior, definido como $h_{t-1}$.

#### Funcionamiento de una red recurrente simple

Para explicar de manera sencilla su funcionamiento, vamos a desenrollar una red recurrente sencilla formada por una única neurona en una secuencia temporal de $T$ pasos, comenzando con el paso anterior al actual (i.e., $t-1$).

<img src="images_1/rnn_simple_explained.png" width="500" data-align="center">

En este ejemplo, se pueden observar tres capas de neuronas:

* **Capa de entrada**, que se corresponde con la información de entrada a la red y que, en el caso de NLP suele ser una palabra, la cual suele ser traducida a un vector numérico mediante un *embedding*. Se indica como $x(t)$.

* **Un número finito de capas densas ocultas**. En este caso mostramos una única capa oculta, pero la información de entrada y el estado anterior podrian ser combinados y pasar por múltiples de ellas antes de ser expulsados. La salida de esta capa será utilizada como entrada de la capa siguiente, representa el "estado" de la red o "memoria". Se indica como $h(t)$.

* **Capa de salida**. La salida de la red para el instante $t$. Por ejemplo, si utilizamos una RNN para predecir el token más probable a partir de una frase tendriamos en cada momento algo de este estilo:

<img src="images_1/rnn_simple_example.jpg" width="700" data-align="center">

Los diferentes elementos matemáticos que se presentan son:
* $x(t)$, $h(t)$ e $y(t)$ se corresponden con la entrada, el estado oculto y la salida de la red, respectivamente.
* $W_{hh}$ es la matriz de peso que conecta las neuronas de la capa oculta consigo mismas simulando el proceso de memoria a corto plazo.
* $W_{xh}$ es la matriz de peso que conecta las neuronas de la capa de entrada con las neuronas de la capa oculta.
* $W_{hy}$ es la matriz de peso que conecta las neuronas de la capa oculta con las neuronas de la capa de salida.

### 1.3.2 - Redes recurrentes complejas

El problema de las RNN simples es que pueden sufrir el problem del "desvanecimiento de gradiente". Esto ocurre porque cuanto mayor sea el número de instantes de tiempo que tengamos, mas "larga" será la red.

El problema del desvanecimiento de gradiente se muestra en el hecho de que las RNNs simples se "olvidan" de información a largo plazo. Por ejemplo, en el ámbito de NLP, **la información del inicio de la frase/párrafo tiende a ser "olvidada"** y solo se centran en las palabras más cercanas a la que esta siendo evaluada en ese momento.

Para paliar este problema, se desarrollaron dos neuronas más complejas:

* *Long Short-Term Memory* (LSTM).
* *Gated Recurrent Unit* (GRU).

----

**Nota:** En esta sección solo vamos comentar brevemente ambas arquitecturas. [**Recomiendo leer el capítulo 12 del libro de FastAI (gratuito)**](https://github.com/fastai/fastbook/blob/master/12_nlp_dive.ipynb)

----

#### Long Short-Term Memory (LSTM)

Las neuronas de tipo ***Long Short-Term Memory*** (LSTM) fueron introducidas por Hochreiter y Schmidhuber en 1997 con el propósito de resolver estos problemas de dependencias a largo plazo.

Este nuevo tipo de neurona cuenta con no uno sino dos estados ocultos:
* El estado oculto $h$ que se encarga de combinar la información de "corto plazo" con el input del token actual.
* El estado oculto $c$ (i.e., *cell state*) que se encarga de almacenar la información de "largo plazo".

<img src="images_1/lstm_diagram.png" width="450" data-align="center">

La primera puerta se llama **la puerta del olvido** (*forget gate*). Dado que es una capa lineal seguida de un sigmoide, su salida consistirá en escalares entre 0 y 1. Multiplicamos este resultado por el estado de la celda para determinar qué información conservar y cuál descartar: los valores más cercanos a 0 se "descartan". Esto le da al LSTM la capacidad de "olvidar" cosas sobre su estado a largo plazo.

La segunda puerta se llama **puerta de entrada** (*input gate*). Cuenta con una "puerta interna", que a veces se denomina **puerta de celda** (*cell gate*), para actualizar el estado de la celda. Similar a la puerta de olvido, la puerta de entrada decide qué elementos del estado de la celda actualizar (valores cercanos a 1) y la puerta de la celda determina cuáles son esos valores de actualización, en el rango de -1 a 1 (tanh). Por ejemplo, es posible que veamos un nuevo pronombre de género, en cuyo caso necesitaremos reemplazar la información sobre el género que la puerta de olvido "eliminó" (poner cerca de 0).

La última puerta es la **puerta de salida** (*output gate*). Determina qué información del estado de la celda usar para generar la salida (para la siguiente palabra). el estado de la celda pasa por un tanh antes de combinarse con la salida sigmoidea de la puerta de salida, y el resultado es el nuevo estado oculto.

#### Gated recurrent units

Las neuronas de tipo LSTM ofrecían grandes resultados, pero presentaban una elevada complejidad. Para simplificar
su funcionamiento, se introdujeron las neuronas ***Gated Recurrent Units*** (GRU), que proporcionaban resultados similares a las neuronas LSTM, aunque con una estructura mucho más sencilla conformada únicamente por dos puertas y sin la necesidad de utilizar dos estados ocultos.

<img src="images_1/gru_diagram.png" width="450" data-align="center">

La **puerta de reinicio** (*reset gate*) funciona de forma similar a la de olvido de la LSTM pero usa el estado oculto del instante anterior $h_{t-1}$.

La **puerta nueva** (*new gate*) funciona de forma similar a la **puerta de entrada** de la LSTM. Se combina la información de entrada $x_{t}$ con el estado oculto del instante anterior $h_{t-1}$ (tras el reinicio).

La **puerta de actualización** (*update gate*) selecciona el "grado de importancia" del estado oculto anterior $h_{t-1}$. Tiene ciertas similitudes con la puerta de reinicio.

## 1.4 - Encoders-Decoders

## 1.5 - Modelos de atención

## 1.6 - Transformers

## 1.7 - Construcción de una red recurrente

### 1.7.1 - Importación de la librería

### 1.7.2 - Carga de datos

### 1.7.3 - Preparación del sistema de codificación de palabras

### 1.7.4 - Creación de la red recurrente

### 1.7.5 - Entrenamiento

### 1.7.6 - Prueba del modelo